# YOLOv8n Object Detection PyTorch Model - Quantization for IMX500

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/pytorch/ptq/pytorch_yolov8n_for_imx500.ipynb)

## Overview

In this tutorial, we will illustrate a basic and quick process of preparing a pre-trained model for deployment using MCT. Specifically, we will demonstrate how to download a pre-trained YOLOv8n model from the MCT Models Library, compress it, and make it deployment-ready using MCT's post-training quantization techniques.

We will use an existing pre-trained YOLOv8n model based on [Ultralytics](https://github.com/ultralytics/ultralytics). The model was slightly adjusted for model quantization. We will quantize the model using MCT post training quantization and evaluate the performance of the floating point model and the quantized model on COCO dataset.


## Summary

In this tutorial we will cover:

1. Post-Training Quantization using MCT of PyTorch object detection model.
2. Data preparation - loading and preprocessing validation and representative datasets from COCO.
3. Accuracy evaluation of the floating-point and the quantized models.

## Setup
### Install the relevant packages

In [ ]:
!pip install -q torch
!pip install onnx
!pip install -q pycocotools
!pip install 'huggingface-hub>=0.21.0'
!pip install --pre 'sony-custom-layers-dev>=0.2.0.dev5'

 Clone a copy of the [MCT](https://github.com/sony/model_optimization) (Model Compression Toolkit) into your current directory. This step ensures that you have access to [MCT Models Garden](https://github.com/sony/model_optimization/tree/main/tutorials/mct_model_garden) folder which contains all the necessary utility functions for this tutorial.
  **It's important to note that we use the most up-to-date MCT code available.**

In [ ]:
!git clone https://github.com/sony/model_optimization.git local_mct
!pip install -r ./local_mct/requirements.txt
import sys
sys.path.insert(0,"./local_mct")

### Download COCO evaluation set

In [ ]:
!wget -nc http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q -o annotations_trainval2017.zip -d ./coco
!echo Done loading annotations
!wget -nc http://images.cocodataset.org/zips/val2017.zip
!unzip -q -o val2017.zip -d ./coco
!echo Done loading val2017 images

## Model Quantization

### Download a Pre-Trained Model 

We begin by loading a pre-trained [YOLOv8n](https://huggingface.co/SSI-DNN/pytorch_yolov8n_640x640_bb_decoding) model. This implementation is based on [Ultralytics](https://github.com/ultralytics/ultralytics) and includes a slightly modified version of yolov8 detection-head (mainly the box decoding part) that was adapted for model quantization. For further insights into the model's implementation details, please refer to [MCT Models Garden - yolov8](https://github.com/sony/model_optimization/tree/main/tutorials/mct_model_garden/models_pytorch/yolov8).  

In [ ]:
from tutorials.mct_model_garden.models_pytorch.yolov8.yolov8 import DetectionModelPyTorch, yaml_load, model_predict
cfg_dict = yaml_load("./local_mct/tutorials/mct_model_garden/models_pytorch/yolov8/yolov8n.yaml", append_filename=True)  # model dict
model = DetectionModelPyTorch.from_pretrained("SSI-DNN/pytorch_yolov8n_640x640_bb_decoding", cfg=cfg_dict)

### Post training quantization using Model Compression Toolkit 

Now, we're all set to use MCT's post-training quantization. To begin, we'll define a representative dataset and proceed with the model quantization. Please note that, for demonstration purposes, we'll use the evaluation dataset as our representative dataset. We'll calibrate the model using 100 representative images, divided into 20 iterations of 'batch_size' images each. 

Additionally, to further compress the model's memory footprint, we will employ the mixed-precision quantization technique. This method allows each layer to be quantized with different precision options: 2, 4, and 8 bits, aligning with the imx500 target platform capabilities.

In [ ]:
import model_compression_toolkit as mct
from tutorials.mct_model_garden.evaluation_metrics.coco_evaluation import coco_dataset_generator
from tutorials.mct_model_garden.models_pytorch.yolov8.yolov8_preprocess import yolov8_preprocess_chw_transpose
from typing import Iterator, Tuple, List

REPRESENTATIVE_DATASET_FOLDER = './coco/val2017/'
REPRESENTATIVE_DATASET_ANNOTATION_FILE = './coco/annotations/instances_val2017.json'
BATCH_SIZE = 4
n_iters = 20

# Load representative dataset
representative_dataset = coco_dataset_generator(dataset_folder=REPRESENTATIVE_DATASET_FOLDER,
                                                annotation_file=REPRESENTATIVE_DATASET_ANNOTATION_FILE,
                                                preprocess=yolov8_preprocess_chw_transpose,
                                                batch_size=BATCH_SIZE)

# Define representative dataset generator
def get_representative_dataset(n_iter: int, dataset_loader: Iterator[Tuple]):
    """
    This function creates a representative dataset generator. The generator yields numpy
        arrays of batches of shape: [Batch, H, W ,C].
    Args:
        n_iter: number of iterations for MCT to calibrate on
    Returns:
        A representative dataset generator
    """       
    def representative_dataset() -> Iterator[List]:
        ds_iter = iter(dataset_loader)
        for _ in range(n_iter):
            yield [next(ds_iter)[0]]

    return representative_dataset

# Get representative dataset generator
representative_dataset_gen = get_representative_dataset(n_iter=n_iters,
                                                        dataset_loader=representative_dataset)

# Set IMX500-v1 TPC
tpc = mct.get_target_platform_capabilities(fw_name="pytorch",
                                           target_platform_name='imx500',
                                           target_platform_version='v1')

# Specify the necessary configuration for mixed precision quantization. To keep the tutorial brief, we'll use a small set of images and omit the hessian metric for mixed precision calculations. It's important to be aware that this choice may impact the resulting accuracy. 
mp_config = mct.core.MixedPrecisionQuantizationConfig(num_of_images=5,
                                                      use_hessian_based_scores=False)
config = mct.core.CoreConfig(mixed_precision_config=mp_config,
                             quantization_config=mct.core.QuantizationConfig(shift_negative_activation_correction=True))

# Define target Resource Utilization for mixed precision weights quantization (75% of 'standard' 8bits quantization)
resource_utilization_data = mct.core.pytorch_resource_utilization_data(in_model=model,
                                                                       representative_data_gen=
                                                                       representative_dataset_gen,
                                                                       core_config=config,
                                                                       target_platform_capabilities=tpc)
resource_utilization = mct.core.ResourceUtilization(weights_memory=resource_utilization_data.weights_memory * 0.75)

# Perform post training quantization
quant_model, _ = mct.ptq.pytorch_post_training_quantization(in_module=model,
                                                            representative_data_gen=
                                                            representative_dataset_gen,
                                                            target_resource_utilization=resource_utilization,
                                                            core_config=config,
                                                            target_platform_capabilities=tpc)
print('Quantized model is ready')

# Wrapped the quantized model with PostProcess NMS.
from tutorials.mct_model_garden.models_pytorch.yolov8.yolov8 import PostProcessWrapper

# Define PostProcess params
score_threshold = 0.001
iou_threshold = 0.7
max_detections = 300

# Get working device
from model_compression_toolkit.core.pytorch.pytorch_device_config import get_working_device
device = get_working_device()

quant_model_pp = PostProcessWrapper(model=quant_model,
                                    score_threshold=score_threshold,
                                    iou_threshold=iou_threshold,
                                    max_detections=max_detections).to(device=device)

### Model Export

Now, we can export the quantized model, ready for deployment, into a `.onnx` format file. Please ensure that the `save_model_path` has been set correctly. 

In [ ]:
mct.exporter.pytorch_export_model(model=quant_model_pp,
                                  save_model_path='./qmodel_pp.onnx',
                                  repr_dataset=representative_dataset_gen)

## Evaluation on COCO dataset

### Floating point model evaluation
Next, we evaluate the floating point model by using `cocoeval` library alongside additional dataset utilities. We can verify the mAP accuracy aligns with that of the original model. 
Note that we set the "batch_size" to 4 and the preprocessing according to [Ultralytics](https://github.com/ultralytics/ultralytics).
Please ensure that the dataset path has been set correctly before running this code cell.

In [ ]:
from tutorials.mct_model_garden.evaluation_metrics.coco_evaluation import coco_evaluate

EVAL_DATASET_FOLDER = './coco/val2017'
EVAL_DATASET_ANNOTATION_FILE = './coco/annotations/instances_val2017.json'
INPUT_RESOLUTION = 640

# Define resizing information to map between the model's output and the original image dimensions
output_resize = {'shape': (INPUT_RESOLUTION, INPUT_RESOLUTION), 'aspect_ratio_preservation': True}

# Wrapped the model with PostProcess NMS.
# Define PostProcess params
score_threshold = 0.001
iou_threshold = 0.7
max_detections = 300

model_pp = PostProcessWrapper(model=model,
                              score_threshold=score_threshold,
                              iou_threshold=iou_threshold,
                              max_detections=max_detections).to(device=device)

# Evaluate the model on coco
eval_results = coco_evaluate(model=model_pp,
                             dataset_folder=EVAL_DATASET_FOLDER,
                             annotation_file=EVAL_DATASET_ANNOTATION_FILE,
                             preprocess=yolov8_preprocess_chw_transpose,
                             output_resize=output_resize,
                             batch_size=BATCH_SIZE,
                             model_inference=model_predict)

# Print float model mAP results
print("Float model mAP: {:.4f}".format(eval_results[0]))

### Quantized model evaluation
Lastly, we can evaluate the performance of the quantized model. There is a slight decrease in performance that can be further mitigated by either expanding the representative dataset or employing MCT's advanced quantization methods, such as GPTQ (Gradient-Based/Enhanced Post Training Quantization).

In [ ]:
# Evaluate the quantized model with PostProcess on coco
eval_results = coco_evaluate(model=quant_model_pp,
                             dataset_folder=EVAL_DATASET_FOLDER,
                             annotation_file=EVAL_DATASET_ANNOTATION_FILE,
                             preprocess=yolov8_preprocess_chw_transpose,
                             output_resize=output_resize,
                             batch_size=BATCH_SIZE,
                             model_inference=model_predict)

# Print quantized model mAP results
print("Quantized model mAP: {:.4f}".format(eval_results[0]))

\
Copyright 2024 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.